# LangChain 1.0 Structured Output

Ref:
https://docs.langchain.com/oss/python/langchain/agents#advanced-concepts

In [0]:
%pip install -U langchain langchain_openai mlflow

%restart_python

## モデルとの接続

In [0]:
from langchain_openai import ChatOpenAI
import mlflow

mlflow.langchain.autolog()

creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
model = ChatOpenAI(
    model="databricks-llama-4-maverick",
    base_url=creds.host + "/serving-endpoints",
    api_key=creds.token,
)

gemma_model = ChatOpenAI(
    model="databricks-gemma-3-12b",
    base_url=creds.host + "/serving-endpoints",
    api_key=creds.token,
)



## エージェントの作成と実行

In [0]:
from pydantic import BaseModel

class WeatherInfo(BaseModel):
    city: str
    weather: str
    probability: int


In [0]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from pprint import pprint


def get_weather(city: str) -> str:
    """指定した都市の天気を取得します。"""
    return f"It's always sunny in {city}!"


agent = create_agent(
    model=model,
    tools=[get_weather],
    response_format=ToolStrategy(WeatherInfo),
)

input = {"messages": [{"role": "user", "content": "SFの天気は？"}]}
context = {"user_role": "expert"}
for event in agent.stream(input, context=context, stream_mode="updates"):
    pprint(event)

# structured output
pprint(event["model"]["structured_response"])


In [0]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model=gemma_model,
    tools=[get_weather],
    response_format=ProviderStrategy(WeatherInfo),
)

input = {"messages": [{"role": "user", "content": "SFの天気は？"}]}
context = {"user_role": "expert"}
for event in agent.stream(input, context=context, stream_mode="updates"):
    pprint(event)

# structured output
pprint(event["model"]["structured_response"])
